# Importing

In [239]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from sklearn.tree import DecisionTreeRegressor

import pmdarima as pm
from pmdarima import auto_arima

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

import warnings 




from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tools.eval_measures import rmse


In [240]:
warnings.filterwarnings("ignore") 

# Load Data

In [241]:
df_nvidia = pd.read_csv('NVDA.csv')
df_amd = pd.read_csv('AMD.csv')

In [242]:
df_nvidia

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,% CHG,CHANGE
0,1/4/2000,0.958333,0.960938,0.901042,0.949219,30048000,-2.670160,-0.026041
1,1/5/2000,0.921875,0.937500,0.904948,0.917969,18835200,-3.292180,-0.031250
2,1/6/2000,0.917969,0.917969,0.822917,0.858073,12048000,-6.524839,-0.059896
3,1/7/2000,0.854167,0.881510,0.841146,0.872396,7118400,1.669205,0.014323
4,1/10/2000,0.875000,0.937500,0.859375,0.901042,23985600,3.283601,0.028646
...,...,...,...,...,...,...,...,...
6110,4/18/2024,849.700012,861.900024,824.020020,846.710022,44726000,0.756833,6.360046
6111,4/19/2024,831.500000,843.239990,756.059998,762.000000,87190500,-10.004608,-84.710022
6112,4/22/2024,781.039978,800.729980,764.000000,795.179993,59634100,4.354330,33.179993
6113,4/23/2024,807.690002,827.690002,802.640015,824.229980,43855900,3.653259,29.049987


In [243]:
df_amd

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,% CHG,CHANGE
0,1/4/2000,15.125,15.500,14.595,14.625,3.15m,1.07%,0.16
1,1/5/2000,14.530,14.690,14.000,14.655,2.87m,0.21%,0.03
2,1/6/2000,15.500,16.000,15.250,16.000,5.74m,9.18%,1.35
3,1/7/2000,15.405,16.405,15.375,16.250,4.27m,1.56%,0.25
4,1/10/2000,16.750,17.750,16.690,17.500,6.46m,7.69%,1.25
...,...,...,...,...,...,...,...,...
6259,4/18/2024,155.510,156.960,152.320,155.080,52.67m,0.69%,1.06
6260,4/19/2024,151.590,154.250,145.290,146.640,71.62m,-5.44%,-8.44
6261,4/22/2024,148.150,149.890,145.630,148.640,49.40m,1.36%,2.00
6262,4/23/2024,151.650,153.500,150.350,152.270,46.05m,2.44%,3.63


# Exploratory Data Analysis

## Convert the date columns to datetime

In [244]:
df_nvidia['DATE'] = pd.to_datetime(df_nvidia['DATE'])
df_amd['DATE'] = pd.to_datetime(df_amd['DATE'])

In [245]:
df_nvidia

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,% CHG,CHANGE
0,2000-01-04,0.958333,0.960938,0.901042,0.949219,30048000,-2.670160,-0.026041
1,2000-01-05,0.921875,0.937500,0.904948,0.917969,18835200,-3.292180,-0.031250
2,2000-01-06,0.917969,0.917969,0.822917,0.858073,12048000,-6.524839,-0.059896
3,2000-01-07,0.854167,0.881510,0.841146,0.872396,7118400,1.669205,0.014323
4,2000-01-10,0.875000,0.937500,0.859375,0.901042,23985600,3.283601,0.028646
...,...,...,...,...,...,...,...,...
6110,2024-04-18,849.700012,861.900024,824.020020,846.710022,44726000,0.756833,6.360046
6111,2024-04-19,831.500000,843.239990,756.059998,762.000000,87190500,-10.004608,-84.710022
6112,2024-04-22,781.039978,800.729980,764.000000,795.179993,59634100,4.354330,33.179993
6113,2024-04-23,807.690002,827.690002,802.640015,824.229980,43855900,3.653259,29.049987


In [246]:
df_amd

,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,% CHG,CHANGE
0,2000-01-04,15.125,15.500,14.595,14.625,3.15m,1.07%,0.16
1,2000-01-05,14.530,14.690,14.000,14.655,2.87m,0.21%,0.03
2,2000-01-06,15.500,16.000,15.250,16.000,5.74m,9.18%,1.35
3,2000-01-07,15.405,16.405,15.375,16.250,4.27m,1.56%,0.25
4,2000-01-10,16.750,17.750,16.690,17.500,6.46m,7.69%,1.25
...,...,...,...,...,...,...,...,...
6259,2024-04-18,155.510,156.960,152.320,155.080,52.67m,0.69%,1.06
6260,2024-04-19,151.590,154.250,145.290,146.640,71.62m,-5.44%,-8.44
6261,2024-04-22,148.150,149.890,145.630,148.640,49.40m,1.36%,2.00
6262,2024-04-23,151.650,153.500,150.350,152.270,46.05m,2.44%,3.63


## get similar dates

In [247]:
common_dates = set(df_nvidia['DATE']).intersection(set(df_amd['DATE']))

## Filter common dates

In [248]:
df_nvidia = df_nvidia[df_nvidia['DATE'].isin(common_dates)]
df_amd = df_amd[df_amd['DATE'].isin(common_dates)]

In [249]:
df_nvidia.shape

(6115, 8)

In [250]:
df_amd.shape

(6115, 8)

## merge data sets

In [251]:
df_merged = pd.merge(df_nvidia, df_amd, left_on='DATE', right_on='DATE')

In [252]:
df_merged

,DATE,OPEN_x,HIGH_x,LOW_x,CLOSE_x,VOLUME_x,% CHG_x,CHANGE_x,OPEN_y,HIGH_y,LOW_y,CLOSE_y,VOLUME_y,% CHG_y,CHANGE_y
0,2000-01-04,0.958333,0.960938,0.901042,0.949219,30048000,-2.670160,-0.026041,15.125,15.500,14.595,14.625,3.15m,1.07%,0.16
1,2000-01-05,0.921875,0.937500,0.904948,0.917969,18835200,-3.292180,-0.031250,14.530,14.690,14.000,14.655,2.87m,0.21%,0.03
2,2000-01-06,0.917969,0.917969,0.822917,0.858073,12048000,-6.524839,-0.059896,15.500,16.000,15.250,16.000,5.74m,9.18%,1.35
3,2000-01-07,0.854167,0.881510,0.841146,0.872396,7118400,1.669205,0.014323,15.405,16.405,15.375,16.250,4.27m,1.56%,0.25
4,2000-01-10,0.875000,0.937500,0.859375,0.901042,23985600,3.283601,0.028646,16.750,17.750,16.690,17.500,6.46m,7.69%,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6110,2024-04-18,849.700012,861.900024,824.020020,846.710022,44726000,0.756833,6.360046,155.510,156.960,152.320,155.080,52.67m,0.69%,1.06
6111,2024-04-19,831.500000,843.239990,756.059998,762.000000,87190500,-10.004608,-84.710022,151.590,154.250,145.290,146.640,71.62m,-5.44%,-8.44
6112,2024-04-22,781.039978,800.729980,764.000000,795.179993,59634100,4.354330,33.179993,148.150,149.890,145.630,148.640,49.40m,1.36%,2.00
6113,2024-04-23,807.690002,827.690002,802.640015,824.229980,43855900,3.653259,29.049987,151.650,153.500,150.350,152.270,46.05m,2.44%,3.63


In [253]:
# Rename columns for clarity
df_merged.rename(columns={
    'OPEN_x': 'Open_nvidia', 'HIGH_x': 'High_nvidia', 'LOW_x': 'Low_nvidia', 'CLOSE_x': 'Close_nvidia', 'VOLUME_x': 'Volume_nvidia',
    'OPEN_y': 'Open_amd', 'HIGH_y': 'High_amd', 'LOW_y': 'Low_amd', 'CLOSE_y': 'Close_amd', 'VOLUME_y': 'Volume_amd',
    '% CHG_x': '% CHG_nvidia', 'CHANGE_x': 'CHANGE_nvidia',
    '% CHG_y': '% CHG_amd', 'CHANGE_y': 'CHANGE_amd'
}, inplace=True)

In [254]:
df_merged

,DATE,Open_nvidia,High_nvidia,Low_nvidia,Close_nvidia,Volume_nvidia,% CHG_nvidia,CHANGE_nvidia,Open_amd,High_amd,Low_amd,Close_amd,Volume_amd,% CHG_amd,CHANGE_amd
0,2000-01-04,0.958333,0.960938,0.901042,0.949219,30048000,-2.670160,-0.026041,15.125,15.500,14.595,14.625,3.15m,1.07%,0.16
1,2000-01-05,0.921875,0.937500,0.904948,0.917969,18835200,-3.292180,-0.031250,14.530,14.690,14.000,14.655,2.87m,0.21%,0.03
2,2000-01-06,0.917969,0.917969,0.822917,0.858073,12048000,-6.524839,-0.059896,15.500,16.000,15.250,16.000,5.74m,9.18%,1.35
3,2000-01-07,0.854167,0.881510,0.841146,0.872396,7118400,1.669205,0.014323,15.405,16.405,15.375,16.250,4.27m,1.56%,0.25
4,2000-01-10,0.875000,0.937500,0.859375,0.901042,23985600,3.283601,0.028646,16.750,17.750,16.690,17.500,6.46m,7.69%,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6110,2024-04-18,849.700012,861.900024,824.020020,846.710022,44726000,0.756833,6.360046,155.510,156.960,152.320,155.080,52.67m,0.69%,1.06
6111,2024-04-19,831.500000,843.239990,756.059998,762.000000,87190500,-10.004608,-84.710022,151.590,154.250,145.290,146.640,71.62m,-5.44%,-8.44
6112,2024-04-22,781.039978,800.729980,764.000000,795.179993,59634100,4.354330,33.179993,148.150,149.890,145.630,148.640,49.40m,1.36%,2.00
6113,2024-04-23,807.690002,827.690002,802.640015,824.229980,43855900,3.653259,29.049987,151.650,153.500,150.350,152.270,46.05m,2.44%,3.63


In [255]:
df_merged.isna().sum()

DATE             0
Open_nvidia      0
High_nvidia      0
Low_nvidia       0
Close_nvidia     0
Volume_nvidia    0
% CHG_nvidia     0
CHANGE_nvidia    0
Open_amd         4
High_amd         4
Low_amd          4
Close_amd        0
Volume_amd       4
% CHG_amd        0
CHANGE_amd       4
dtype: int64

In [256]:
# drop na
df_merged.dropna(subset=['Open_amd', 'High_amd', 'Low_amd', 'Volume_amd', 'CHANGE_amd'], inplace=True)

In [257]:
df_merged.isna().sum()

DATE             0
Open_nvidia      0
High_nvidia      0
Low_nvidia       0
Close_nvidia     0
Volume_nvidia    0
% CHG_nvidia     0
CHANGE_nvidia    0
Open_amd         0
High_amd         0
Low_amd          0
Close_amd        0
Volume_amd       0
% CHG_amd        0
CHANGE_amd       0
dtype: int64

In [258]:
df_merged.isnull().sum()

DATE             0
Open_nvidia      0
High_nvidia      0
Low_nvidia       0
Close_nvidia     0
Volume_nvidia    0
% CHG_nvidia     0
CHANGE_nvidia    0
Open_amd         0
High_amd         0
Low_amd          0
Close_amd        0
Volume_amd       0
% CHG_amd        0
CHANGE_amd       0
dtype: int64

In [259]:
# Re-format the date column and set it as index
df_merged['DATE'] = pd.to_datetime(df_merged['DATE'], format = '%Y/%m/%d')
df_merged.set_index('DATE', inplace = True)
df_merged.sort_index(inplace = True)

In [260]:
df_merged

,Open_nvidia,High_nvidia,Low_nvidia,Close_nvidia,Volume_nvidia,% CHG_nvidia,CHANGE_nvidia,Open_amd,High_amd,Low_amd,Close_amd,Volume_amd,% CHG_amd,CHANGE_amd
DATE,,,,,,,,,,,,,,
2000-01-04,0.958333,0.960938,0.901042,0.949219,30048000,-2.670160,-0.026041,15.125,15.500,14.595,14.625,3.15m,1.07%,0.16
2000-01-05,0.921875,0.937500,0.904948,0.917969,18835200,-3.292180,-0.031250,14.530,14.690,14.000,14.655,2.87m,0.21%,0.03
2000-01-06,0.917969,0.917969,0.822917,0.858073,12048000,-6.524839,-0.059896,15.500,16.000,15.250,16.000,5.74m,9.18%,1.35
2000-01-07,0.854167,0.881510,0.841146,0.872396,7118400,1.669205,0.014323,15.405,16.405,15.375,16.250,4.27m,1.56%,0.25
2000-01-10,0.875000,0.937500,0.859375,0.901042,23985600,3.283601,0.028646,16.750,17.750,16.690,17.500,6.46m,7.69%,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-18,849.700012,861.900024,824.020020,846.710022,44726000,0.756833,6.360046,155.510,156.960,152.320,155.080,52.67m,0.69%,1.06
2024-04-19,831.500000,843.239990,756.059998,762.000000,87190500,-10.004608,-84.710022,151.590,154.250,145.290,146.640,71.62m,-5.44%,-8.44
2024-04-22,781.039978,800.729980,764.000000,795.179993,59634100,4.354330,33.179993,148.150,149.890,145.630,148.640,49.40m,1.36%,2.00


In [261]:
# Set the dataset frequency to be (D)aily data
df_merged = df_merged.asfreq('D', method = 'bfill') 

In [262]:
df_merged

,Open_nvidia,High_nvidia,Low_nvidia,Close_nvidia,Volume_nvidia,% CHG_nvidia,CHANGE_nvidia,Open_amd,High_amd,Low_amd,Close_amd,Volume_amd,% CHG_amd,CHANGE_amd
DATE,,,,,,,,,,,,,,
2000-01-04,0.958333,0.960938,0.901042,0.949219,30048000,-2.670160,-0.026041,15.125,15.500,14.595,14.625,3.15m,1.07%,0.16
2000-01-05,0.921875,0.937500,0.904948,0.917969,18835200,-3.292180,-0.031250,14.530,14.690,14.000,14.655,2.87m,0.21%,0.03
2000-01-06,0.917969,0.917969,0.822917,0.858073,12048000,-6.524839,-0.059896,15.500,16.000,15.250,16.000,5.74m,9.18%,1.35
2000-01-07,0.854167,0.881510,0.841146,0.872396,7118400,1.669205,0.014323,15.405,16.405,15.375,16.250,4.27m,1.56%,0.25
2000-01-08,0.875000,0.937500,0.859375,0.901042,23985600,3.283601,0.028646,16.750,17.750,16.690,17.500,6.46m,7.69%,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-20,781.039978,800.729980,764.000000,795.179993,59634100,4.354330,33.179993,148.150,149.890,145.630,148.640,49.40m,1.36%,2.00
2024-04-21,781.039978,800.729980,764.000000,795.179993,59634100,4.354330,33.179993,148.150,149.890,145.630,148.640,49.40m,1.36%,2.00
2024-04-22,781.039978,800.729980,764.000000,795.179993,59634100,4.354330,33.179993,148.150,149.890,145.630,148.640,49.40m,1.36%,2.00


In [263]:
df_merged.isnull().sum()

Open_nvidia      0
High_nvidia      0
Low_nvidia       0
Close_nvidia     0
Volume_nvidia    0
% CHG_nvidia     0
CHANGE_nvidia    0
Open_amd         0
High_amd         0
Low_amd          0
Close_amd        0
Volume_amd       0
% CHG_amd        0
CHANGE_amd       0
dtype: int64

## Defining the train-test periods.

In [264]:
# Determine the entire date range
total_dates = df_merged.index
train_size = int(len(total_dates) * 0.8)  # 80% for training
train_dates = total_dates[:train_size]
test_dates = total_dates[train_size:]

In [265]:
train_start = train_dates[0]
train_end = train_dates[-1]
test_start = test_dates[0]
test_end = test_dates[-1]
forecast_start = test_end + pd.Timedelta(days=1)
forecast_end = forecast_start + pd.Timedelta(days=30)

In [266]:
print(f"Train Start: {train_start}")
print(f"Train End: {train_end}")
print(f"Test Start: {test_start}")
print(f"Test End: {test_end}")
print(f"Forecast Start: {forecast_start}")
print(f"Forecast End: {forecast_end}")

Train Start: 2000-01-04 00:00:00
Train End: 2019-06-14 00:00:00
Test Start: 2019-06-15 00:00:00
Test End: 2024-04-24 00:00:00
Forecast Start: 2024-04-25 00:00:00
Forecast End: 2024-05-25 00:00:00


# Visualize

In [267]:
# Create a Plotly figure
fig = go.Figure()

# Add the training data trace
fig.add_trace(go.Scatter(
    x=df_merged.loc[train_start:train_end].index,
    y=df_merged.loc[train_start:train_end]['Close_nvidia'],
    mode='lines',
    name='Train'
))

# Add the testing data trace
fig.add_trace(go.Scatter(
    x=df_merged.loc[test_start:test_end].index,
    y=df_merged.loc[test_start:test_end]['Close_nvidia'],
    mode='lines',
    name='Test'
))

# Customize the layout
fig.update_layout(
    title='Nvidia Stock Prices: Train and Test Periods',
    xaxis_title='Date',
    yaxis_title='Close Price',
    legend_title='Periods',
    template='plotly_dark'
)

# Show the plot
fig.show()

In [268]:
# majority of our data is used for training

# Simple Model

In [269]:
print(df_merged.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8878 entries, 2000-01-04 to 2024-04-24
Freq: D
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Open_nvidia    8878 non-null   float64
 1   High_nvidia    8878 non-null   float64
 2   Low_nvidia     8878 non-null   float64
 3   Close_nvidia   8878 non-null   float64
 4   Volume_nvidia  8878 non-null   int64  
 5   % CHG_nvidia   8878 non-null   float64
 6   CHANGE_nvidia  8878 non-null   float64
 7   Open_amd       8878 non-null   float64
 8   High_amd       8878 non-null   float64
 9   Low_amd        8878 non-null   float64
 10  Close_amd      8878 non-null   float64
 11  Volume_amd     8878 non-null   object 
 12  % CHG_amd      8878 non-null   object 
 13  CHANGE_amd     8878 non-null   float64
dtypes: float64(11), int64(1), object(2)
memory usage: 1.3+ MB
None


In [270]:
# Define the forecaster
forecaster = ForecasterAutoreg(
    regressor=DecisionTreeRegressor(random_state=123),
    lags=30
)

# Fit the model using train data
forecaster.fit(y=df_merged.loc[train_start:train_end]['Close_nvidia'])

# Predict the test period
predicted_test = forecaster.predict(steps=len(df_merged.loc[test_start:test_end]['Close_nvidia']))

# Print the predictions
print(predicted_test)

2019-06-15    36.689999
2019-06-16    37.064999
2019-06-17    37.930000
2019-06-18    37.467499
2019-06-19    38.092499
                ...    
2024-04-20    38.970001
2024-04-21    39.334999
2024-04-22    39.130001
2024-04-23    39.334999
2024-04-24    39.130001
Freq: D, Name: pred, Length: 1776, dtype: float64
